<a href="https://colab.research.google.com/github/mc-robinson/validating_validation_supp_info/blob/master/Mayr_reanalysis/Mayr_data_analysis/colab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
import altair as alt
alt.renderers.enable('colab')

RendererRegistry.enable('colab')

# Load Data #

In [4]:
!git clone https://github.com/mc-robinson/validating_validation_supp_info.git

Cloning into 'validating_validation_supp_info'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 142 (delta 65), reused 65 (delta 13), pack-reused 0
Receiving objects: 100% (142/142), 11.43 MiB | 18.00 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [5]:
cd validating_validation_supp_info/Mayr_reanalysis/Mayr_data_analysis/

/content/validating_validation_supp_info/Mayr_reanalysis/Mayr_data_analysis


In [6]:
ls ecfp6/

conv.csv*    fold_info.csv               lstm.csv*  sea.csv*
dnn.csv      knn.csv*                    nb.csv*    svm.csv*
dnn_prc.csv  labelsWeakHard.targetNames  rf.csv*    weave.csv*


In [0]:
fold_info_df = pd.read_csv('./ecfp6/fold_info.csv')

fold_info_df['n_actives0'] = round(fold_info_df['test_size0']*fold_info_df['test_imb0'])
fold_info_df['n_inactives0'] = round(fold_info_df['test_size0']*(1-fold_info_df['test_imb0']))

fold_info_df['n_actives1'] = round(fold_info_df['test_size1']*fold_info_df['test_imb1'])
fold_info_df['n_inactives1'] = round(fold_info_df['test_size1']*(1-fold_info_df['test_imb1']))

fold_info_df['n_actives2'] = round(fold_info_df['test_size2']*fold_info_df['test_imb2'])
fold_info_df['n_inactives2'] = round(fold_info_df['test_size2']*(1-fold_info_df['test_imb2']))

fold_info_df.head()

,train_size0,train_size1,train_size2,test_size0,test_size1,test_size2,train_imb0,train_imb1,train_imb2,test_imb0,test_imb1,test_imb2,n_actives0,n_inactives0,n_actives1,n_inactives1,n_actives2,n_inactives2
0,24,32,24,16,8,16,0.833333,0.843750,0.875000,0.875000,0.875000,0.812500,14.0,2.0,7.0,1.0,13.0,3.0
1,9,42,39,36,3,6,0.555556,0.500000,0.512821,0.500000,0.666667,0.500000,18.0,18.0,2.0,1.0,3.0,3.0
2,53,30,51,14,37,16,0.773585,0.600000,0.843137,0.714286,0.891892,0.500000,10.0,4.0,33.0,4.0,8.0,8.0
3,85,69,36,10,26,59,0.929412,0.971014,0.833333,0.900000,0.807692,0.983051,9.0,1.0,21.0,5.0,58.0,1.0
4,52,37,59,22,37,15,0.865385,0.891892,0.847458,0.863636,0.837838,0.933333,19.0,3.0,31.0,6.0,14.0,1.0


In [0]:
def classical_AUC_CI(AUC, n_actives, n_inactives):
    '''
    Function to compute confidence intervals for AUC values
    Uses the form provided in Nicholls (2014)
    Note that returned intervals are often asymmetric.
    '''
    import scipy.stats
    
    var_active = ((AUC**2)*(1-AUC)/(1+AUC))/n_actives
    var_inactive = (AUC*((1-AUC)**2)/(2-AUC))/n_inactives
    
    df_eff = (
        (((var_active/n_actives)+(var_inactive/n_inactives))**2) / 
        (
            (((var_active/n_actives)**2)/(n_actives-1)) +
            (((var_inactive/n_inactives)**2)/(n_inactives-1))
        )
    )
    
    std_error = np.sqrt(var_active+var_inactive)
    
    # transform to new space where continuous
    logit_auc = np.log(AUC/(1-AUC))
    logit_std_error = std_error * (1/(AUC*(1-AUC)))
    
    t_95 = scipy.stats.t.ppf(0.975,df_eff)
    logit_lower_limit = logit_auc - t_95*logit_std_error
    logit_upper_limit = logit_auc + t_95*logit_std_error
    
    lower_limit = 1/(1+np.exp(-1*logit_lower_limit))
    upper_limit = 1/(1+np.exp(-1*logit_upper_limit))
    
    return lower_limit, upper_limit, std_error
    #return std_error

In [0]:
fnn_df = pd.read_csv('./ecfp6/dnn.csv')
fnn_df = fnn_df[['ECFP6: fold0', 'ECFP6: fold1', 'ECFP6: fold2']]
fnn_df = fnn_df.rename(columns={'ECFP6: fold0': "fold0",
                       'ECFP6: fold1': "fold1",
                       'ECFP6: fold2': "fold2",})
fnn_df['mean_auc'] = fnn_df[['fold0', 'fold1', 'fold2']].mean(axis=1)
fnn_df['std_auc'] = fnn_df[['fold0', 'fold1', 'fold2']].std(axis=1)

fnn_df['std_error0'] = classical_AUC_CI(fnn_df['fold0'],
                                        fold_info_df['n_actives0'],
                                        fold_info_df['n_inactives0'])[-1]
fnn_df['std_error1'] = classical_AUC_CI(fnn_df['fold1'],
                                        fold_info_df['n_actives1'],
                                        fold_info_df['n_inactives1'])[-1]
fnn_df['std_error2'] = classical_AUC_CI(fnn_df['fold2'],
                                        fold_info_df['n_actives2'],
                                        fold_info_df['n_inactives2'])[-1]

fnn_df.head()

/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater
  cond = logical_and(cond, (asarray(arg) > 0))
/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


,fold0,fold1,fold2,mean_auc,std_auc,std_error0,std_error1,std_error2
0,0.785714,1.000000,0.948718,0.911477,0.111892,0.141942,0.000000,0.051118
1,0.691358,0.000000,0.555556,0.415638,0.366301,0.087416,0.000000,0.233921
2,0.675000,0.893939,0.562500,0.710480,0.168544,0.149307,0.060241,0.142956
3,1.000000,0.819048,1.000000,0.939683,0.104473,0.000000,0.087861,0.000000
4,0.842105,1.000000,1.000000,0.947368,0.091161,0.096140,0.000000,0.000000


In [0]:

knn_df = pd.read_csv('./ecfp6/knn.csv')
knn_df = knn_df[['fold0', 'fold1', 'fold2']]
knn_df['mean_auc'] = knn_df[['fold0', 'fold1', 'fold2']].mean(axis=1)
knn_df['std_auc'] = knn_df[['fold0', 'fold1', 'fold2']].std(axis=1)

knn_df['std_error0'] = classical_AUC_CI(knn_df['fold0'],
                                        fold_info_df['n_actives0'],
                                        fold_info_df['n_inactives0'])[-1]
knn_df['std_error1'] = classical_AUC_CI(knn_df['fold1'],
                                        fold_info_df['n_actives1'],
                                        fold_info_df['n_inactives1'])[-1]
knn_df['std_error2'] = classical_AUC_CI(knn_df['fold2'],
                                        fold_info_df['n_actives2'],
                                        fold_info_df['n_inactives2'])[-1]

knn_df.head()

/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater
  cond = logical_and(cond, (asarray(arg) > 0))


,fold0,fold1,fold2,mean_auc,std_auc,std_error0,std_error1,std_error2
0,0.785714,0.428571,0.820513,0.678266,0.216941,0.141942,0.315515,0.112143
1,0.651235,0.000000,0.444444,0.365226,0.332766,0.090771,0.000000,0.233921
2,0.750000,0.643939,0.296875,0.563605,0.237004,0.131950,0.133312,0.129403
3,0.666667,0.685714,0.982759,0.778380,0.177253,0.255797,0.120339,0.020784
4,0.912281,0.903226,1.000000,0.938502,0.053451,0.064502,0.051221,0.000000


In [0]:

nb_df = pd.read_csv('./ecfp6/nb.csv')
nb_df = nb_df[['fold0', 'fold1', 'fold2']]
nb_df['mean_auc'] = nb_df[['fold0', 'fold1', 'fold2']].mean(axis=1)
nb_df['std_auc'] = nb_df[['fold0', 'fold1', 'fold2']].std(axis=1)

nb_df['std_error0'] = classical_AUC_CI(nb_df['fold0'],
                                        fold_info_df['n_actives0'],
                                        fold_info_df['n_inactives0'])[-1]
nb_df['std_error1'] = classical_AUC_CI(nb_df['fold1'],
                                        fold_info_df['n_actives1'],
                                        fold_info_df['n_inactives1'])[-1]
nb_df['std_error2'] = classical_AUC_CI(nb_df['fold2'],
                                        fold_info_df['n_actives2'],
                                        fold_info_df['n_inactives2'])[-1]

nb_df.head()

/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater
  cond = logical_and(cond, (asarray(arg) > 0))


,fold0,fold1,fold2,mean_auc,std_auc,std_error0,std_error1,std_error2
0,0.714286,0.142857,0.641026,0.499389,0.310931,0.169557,0.242284,0.164851
1,0.956790,0.000000,0.222222,0.393004,0.500736,0.034903,0.000000,0.188894
2,0.425000,0.712121,0.234375,0.457165,0.240492,0.172025,0.118491,0.118260
3,1.000000,0.723810,0.000000,0.574603,0.516427,0.000000,0.112472,0.000000
4,0.929825,1.000000,1.000000,0.976608,0.040516,0.055506,0.000000,0.000000


In [0]:
rf_df = pd.read_csv('./ecfp6/rf.csv')
rf_df = rf_df[['fold0', 'fold1', 'fold2']]
rf_df['mean_auc'] = rf_df[['fold0', 'fold1', 'fold2']].mean(axis=1)
rf_df['std_auc'] = rf_df[['fold0', 'fold1', 'fold2']].std(axis=1)

rf_df['std_error0'] = classical_AUC_CI(rf_df['fold0'],
                                        fold_info_df['n_actives0'],
                                        fold_info_df['n_inactives0'])[-1]
rf_df['std_error1'] = classical_AUC_CI(rf_df['fold1'],
                                        fold_info_df['n_actives1'],
                                        fold_info_df['n_inactives1'])[-1]
rf_df['std_error2'] = classical_AUC_CI(rf_df['fold2'],
                                        fold_info_df['n_actives2'],
                                        fold_info_df['n_inactives2'])[-1]

rf_df.head()

/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater
  cond = logical_and(cond, (asarray(arg) > 0))


,fold0,fold1,fold2,mean_auc,std_auc,std_error0,std_error1,std_error2
0,0.464286,0.571429,0.410256,0.481990,0.082032,0.221394,0.293584,0.187969
1,0.500000,0.000000,0.666667,0.388889,0.346944,0.096225,0.000000,0.219427
2,0.650000,0.712121,0.593750,0.651957,0.059210,0.153972,0.118491,0.141221
3,1.000000,0.904762,1.000000,0.968254,0.054986,0.000000,0.058716,0.000000
4,0.982456,1.000000,1.000000,0.994152,0.010129,0.023423,0.000000,0.000000


In [0]:
svm_df = pd.read_csv('./ecfp6/svm.csv')
svm_df = svm_df[['fold0', 'fold1', 'fold2']]
svm_df['mean_auc'] = svm_df[['fold0', 'fold1', 'fold2']].mean(axis=1)
svm_df['std_auc'] = svm_df[['fold0', 'fold1', 'fold2']].std(axis=1)

svm_df['std_error0'] = classical_AUC_CI(svm_df['fold0'],
                                        fold_info_df['n_actives0'],
                                        fold_info_df['n_inactives0'])[-1]
svm_df['std_error1'] = classical_AUC_CI(svm_df['fold1'],
                                        fold_info_df['n_actives1'],
                                        fold_info_df['n_inactives1'])[-1]
svm_df['std_error2'] = classical_AUC_CI(svm_df['fold2'],
                                        fold_info_df['n_actives2'],
                                        fold_info_df['n_inactives2'])[-1]

svm_df.head()

/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater
  cond = logical_and(cond, (asarray(arg) > 0))
/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


,fold0,fold1,fold2,mean_auc,std_auc,std_error0,std_error1,std_error2
0,0.535714,0.714286,0.487179,0.579060,0.119597,0.213610,0.239790,0.185811
1,0.867284,0.000000,1.000000,0.622428,0.543108,0.060986,0.000000,0.000000
2,0.750000,0.803030,0.343750,0.632260,0.251260,0.131950,0.092962,0.135596
3,1.000000,0.628571,0.775862,0.801478,0.187035,0.000000,0.130143,0.182074
4,0.964912,1.000000,1.000000,0.988304,0.020258,0.035463,0.000000,0.000000


In [0]:

gc_df = pd.read_csv('./ecfp6/conv.csv')
gc_df = gc_df[['fold0', 'fold1', 'fold2']]
gc_df['mean_auc'] = gc_df[['fold0', 'fold1', 'fold2']].mean(axis=1)
gc_df['std_auc'] = gc_df[['fold0', 'fold1', 'fold2']].std(axis=1)

gc_df['std_error0'] = classical_AUC_CI(gc_df['fold0'],
                                        fold_info_df['n_actives0'],
                                        fold_info_df['n_inactives0'])[-1]
gc_df['std_error1'] = classical_AUC_CI(gc_df['fold1'],
                                        fold_info_df['n_actives1'],
                                        fold_info_df['n_inactives1'])[-1]
gc_df['std_error2'] = classical_AUC_CI(gc_df['fold2'],
                                        fold_info_df['n_actives2'],
                                        fold_info_df['n_inactives2'])[-1]

gc_df.head()

/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater
  cond = logical_and(cond, (asarray(arg) > 0))
/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


,fold0,fold1,fold2,mean_auc,std_auc,std_error0,std_error1,std_error2
0,0.375000,0.571429,0.615385,0.520604,0.127998,0.222796,0.293584,0.169857
1,0.787037,0.250000,0.777778,0.604938,0.307420,0.075735,0.314813,0.188894
2,0.787500,0.833333,0.414062,0.678299,0.229980,0.121262,0.082906,0.141721
3,1.000000,0.876190,1.000000,0.958730,0.071481,0.000000,0.069301,0.000000
4,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000


In [0]:
weave_df = pd.read_csv('./ecfp6/weave.csv')
weave_df = weave_df[['fold0', 'fold1', 'fold2']]
weave_df['mean_auc'] = weave_df[['fold0', 'fold1', 'fold2']].mean(axis=1)
weave_df['std_auc'] = weave_df[['fold0', 'fold1', 'fold2']].std(axis=1)

weave_df['std_error0'] = classical_AUC_CI(weave_df['fold0'],
                                        fold_info_df['n_actives0'],
                                        fold_info_df['n_inactives0'])[-1]
weave_df['std_error1'] = classical_AUC_CI(weave_df['fold1'],
                                        fold_info_df['n_actives1'],
                                        fold_info_df['n_inactives1'])[-1]
weave_df['std_error2'] = classical_AUC_CI(weave_df['fold2'],
                                        fold_info_df['n_actives2'],
                                        fold_info_df['n_inactives2'])[-1]

weave_df.head()

/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater
  cond = logical_and(cond, (asarray(arg) > 0))
/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


,fold0,fold1,fold2,mean_auc,std_auc,std_error0,std_error1,std_error2
0,0.250000,0.714286,0.692308,0.552198,0.261942,0.207020,0.239790,0.153005
1,0.475309,0.000000,0.777778,0.417695,0.392077,0.096082,0.000000,0.188894
2,0.700000,0.886364,0.687500,0.757955,0.111381,0.144092,0.063274,0.131663
3,1.000000,0.790476,0.965517,0.918664,0.112345,0.000000,0.096021,0.037306
4,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000


In [0]:

lstm_df = pd.read_csv('./ecfp6/lstm.csv')
lstm_df = lstm_df[['fold0', 'fold1', 'fold2']]
lstm_df['mean_auc'] = lstm_df[['fold0', 'fold1', 'fold2']].mean(axis=1)
lstm_df['std_auc'] = lstm_df[['fold0', 'fold1', 'fold2']].std(axis=1)

lstm_df['std_error0'] = classical_AUC_CI(lstm_df['fold0'],
                                        fold_info_df['n_actives0'],
                                        fold_info_df['n_inactives0'])[-1]
lstm_df['std_error1'] = classical_AUC_CI(lstm_df['fold1'],
                                        fold_info_df['n_actives1'],
                                        fold_info_df['n_inactives1'])[-1]
lstm_df['std_error2'] = classical_AUC_CI(lstm_df['fold2'],
                                        fold_info_df['n_actives2'],
                                        fold_info_df['n_inactives2'])[-1]

lstm_df.head()

/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
/Users/matthewrobinson/anaconda/envs/mlbook/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater
  cond = logical_and(cond, (asarray(arg) > 0))


,fold0,fold1,fold2,mean_auc,std_auc,std_error0,std_error1,std_error2
0,0.535714,0.428571,0.897436,0.620574,0.245681,0.213610,0.315515,0.078759
1,0.540123,0.500000,0.333333,0.457819,0.109658,0.095846,0.353553,0.219427
2,0.575000,0.856061,0.609375,0.680145,0.153314,0.164750,0.074822,0.140087
3,1.000000,0.742857,1.000000,0.914286,0.148461,0.000000,0.108127,0.000000
4,0.929825,1.000000,1.000000,0.976608,0.040516,0.055506,0.000000,0.000000


In [0]:
canonical_order = pd.read_csv('./ecfp6/nb.csv').iloc[:,0]
canonical_order.head()

0    CHEMBL1614338
1    CHEMBL1243971
2    CHEMBL1614044
3    CHEMBL2045986
4     CHEMBL618396
Name: Unnamed: 0, dtype: object

In [0]:

all_df_dict = {}

all_df_dict['CHEMBL'] = list(canonical_order) + list(canonical_order) + list(canonical_order)
all_df_dict['fold'] = ['fold0']*1310 + ['fold1']*1310 + ['fold2']*1310
all_df_dict['train_imb'] = list(fold_info_df['train_imb0']) + \
                           list(fold_info_df['train_imb1']) + list(fold_info_df['train_imb2'])
all_df_dict['test_imb'] = list(fold_info_df['test_imb0']) + \
                          list(fold_info_df['test_imb1']) + list(fold_info_df['test_imb2'])
all_df_dict['train_size'] = list(fold_info_df['train_size0']) + \
                           list(fold_info_df['train_size1']) + list(fold_info_df['train_size2'])
all_df_dict['test_size'] = list(fold_info_df['test_size0']) + \
                          list(fold_info_df['test_size1']) + list(fold_info_df['test_size2'])
all_df_dict['knn_auc'] = list(knn_df['fold0']) + \
                          list(knn_df['fold1']) + list(knn_df['fold2'])
all_df_dict['nb_auc'] = list(nb_df['fold0']) + \
                          list(nb_df['fold1']) + list(nb_df['fold2'])
all_df_dict['fnn_auc'] = list(fnn_df['fold0']) + \
                          list(fnn_df['fold1']) + list(fnn_df['fold2'])
all_df_dict['svm_auc'] = list(svm_df['fold0']) + \
                          list(svm_df['fold1']) + list(svm_df['fold2'])
all_df_dict['rf_auc'] = list(rf_df['fold0']) + \
                          list(rf_df['fold1']) + list(rf_df['fold2'])
all_df_dict['gc_auc'] = list(gc_df['fold0']) + \
                          list(gc_df['fold1']) + list(gc_df['fold2'])
all_df_dict['weave_auc'] = list(weave_df['fold0']) + \
                          list(weave_df['fold1']) + list(weave_df['fold2'])
all_df_dict['lstm_auc'] = list(lstm_df['fold0']) + \
                          list(lstm_df['fold1']) + list(lstm_df['fold2'])

all_df = pd.DataFrame(all_df_dict)

In [0]:
all_df['winner'] = 'TIE'
all_df['mean_auc'] = 0

for index, row in all_df.iterrows():
    
    knn_score = row['knn_auc']
    nb_score = row['nb_auc']
    fnn_score = row['fnn_auc']
    svm_score = row['svm_auc']
    rf_score = row['rf_auc']
    gc_score = row['gc_auc']
    weave_score = row['weave_auc']
    lstm_score = row['lstm_auc']
    
    top_score = np.max(np.array([knn_score, nb_score, fnn_score,
                       svm_score, rf_score, gc_score,
                       weave_score, lstm_score]))
    
    winner = 'TIE'
    num_winners = 0
    if knn_score == top_score:
        winner = 'KNN'
        num_winners += 1
    if nb_score == top_score:
        winner = 'NB'
        num_winners += 1
    if fnn_score == top_score:
        winner = 'FNN'
        num_winners += 1
    if svm_score == top_score:
        winner = 'SVM'
        num_winners += 1
    if rf_score == top_score:
        winner = 'RF'
        num_winners += 1
    if gc_score == top_score:
        winner = 'GC'
        num_winners += 1
    if weave_score == top_score:
        winner = 'WEAVE'
        num_winners += 1
    if lstm_score == top_score:
        winner = 'LSTM'
        num_winners += 1
        
    if num_winners > 1:
        winner = 'TIE'
    
    all_df.loc[index,'winner'] = winner
    all_df.loc[index,'avg_auc'] = np.mean(np.array([knn_score, nb_score, fnn_score,
                       svm_score, rf_score, gc_score,
                       weave_score, lstm_score]))

# Make Plots #

In [0]:
all_df.head()

,CHEMBL,fold,train_imb,test_imb,train_size,test_size,knn_auc,nb_auc,fnn_auc,svm_auc,rf_auc,gc_auc,weave_auc,lstm_auc,winner,mean_auc,avg_auc
0,CHEMBL1614338,fold0,0.833333,0.875000,24,16,0.785714,0.714286,0.785714,0.535714,0.464286,0.375000,0.250000,0.535714,FNN,0,0.555804
1,CHEMBL1243971,fold0,0.555556,0.500000,9,36,0.651235,0.956790,0.691358,0.867284,0.500000,0.787037,0.475309,0.540123,NB,0,0.683642
2,CHEMBL1614044,fold0,0.773585,0.714286,53,14,0.750000,0.425000,0.675000,0.750000,0.650000,0.787500,0.700000,0.575000,GC,0,0.664062
3,CHEMBL2045986,fold0,0.929412,0.900000,85,10,0.666667,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,TIE,0,0.958333
4,CHEMBL618396,fold0,0.865385,0.863636,52,22,0.912281,0.929825,0.842105,0.964912,0.982456,1.000000,1.000000,0.929825,TIE,0,0.945175


In [0]:
model1 = 'nb' #@param {type:"string"}
model2 = 'knn'  #@param {type:"string"}

if model2 is not '':

    source = all_df

    points = alt.Chart(source).mark_circle().encode(
        alt.X(model1+'_auc'),
        alt.Y(model2+'_auc'),
        size='test_size',
        #color='color:Q',
        tooltip=['CHEMBL', 'fold', 'train_size', 'train_imb', 'test_size', 'test_imb' ]
    ).interactive()

    poly_data = pd.DataFrame({
        'x': list(np.linspace(0,1,100)),
        'y': list(np.linspace(0,1,100)),
    })
    line = alt.Chart(poly_data).mark_line().encode(
        x=alt.X('x',title=f'{model1} AUC-ROC'),
        y=alt.Y('y',title=f'{model2} AUC-ROC'),
        color=alt.value("#FFAA00")
    )
    
else:
    source = all_df

    points = alt.Chart(source).mark_circle().encode(
        alt.X('fnn_auc'),
        alt.Y('knn_auc'),
        size='test_size',
        #color='color:Q',
        tooltip=['CHEMBL', 'fold', 'train_size', 'train_imb', 'test_size', 'test_imb' ]
    ).interactive()

    poly_data = pd.DataFrame({
        'x': list(np.linspace(0,1,100)),
        'y': list(np.linspace(0,1,100)),
    })
    line = alt.Chart(poly_data).mark_line().encode(
        x=alt.X('x',title='FNN AUC-ROC'),
        y=alt.Y('y',title='KNN AUC-ROC'),
        color=alt.value("#FFAA00")
    )
    
points+line

LayerChart({
  layer: [Chart({
    data:              CHEMBL   fold  train_imb  ...  winner  mean_auc   avg_auc
    0     CHEMBL1614338  fold0   0.833333  ...     FNN         0  0.555804
    1     CHEMBL1243971  fold0   0.555556  ...      NB         0  0.683642
    2     CHEMBL1614044  fold0   0.773585  ...      GC         0  0.664062
    3     CHEMBL2045986  fold0   0.929412  ...     TIE         0  0.958333
    4      CHEMBL618396  fold0   0.865385  ...     TIE         0  0.945175
    5     CHEMBL3079731  fold0   0.925000  ...      NB         0  0.697838
    6     CHEMBL1794498  fold0   0.442623  ...   WEAVE         0  0.902500
    7     CHEMBL2091156  fold0   0.815789  ...   WEAVE         0  0.870833
    8     CHEMBL2363253  fold0   0.571429  ...     SVM         0  0.657738
    9     CHEMBL1738192  fold0   0.180556  ...      GC         0  0.631250
    10    CHEMBL1614154  fold0   0.898551  ...     TIE         0  0.898990
    11    CHEMBL1964055  fold0   0.892308  ...    LSTM         0  0.564453
    12    CHEMBL2363800  fold0   0.368421  ...     KNN         0  0.640625
    13    CHEMBL1794420  fold0   0.459016  ...     TIE         0  0.842172
    14    CHEMBL2363263  fold0   0.489796  ...     SVM         0  0.623843
    15    CHEMBL2363552  fold0   0.352941  ...     KNN         0  0.653912
    16    CHEMBL2363168  fold0   0.346154  ...    LSTM         0  0.600000
    17    CHEMBL3214806  fold0   0.880597  ...      RF         0  0.825893
    18    CHEMBL1794455  fold0   0.475410  ...   WEAVE         0  0.895363
    19    CHEMBL1613998  fold0   0.837838  ...     KNN         0  0.706897
    20    CHEMBL1614056  fold0   0.161765  ...      GC         0  0.737500
    21    CHEMBL1613903  fold0   0.441176  ...     FNN         0  0.560150
    22    CHEMBL2363184  fold0   0.203125  ...      NB         0  0.601563
    23    CHEMBL1738619  fold0   0.208333  ...      GC         0  0.740741
    24    CHEMBL2363227  fold0   0.333333  ...     FNN         0  0.647917
    25    CHEMBL1738514  fold0   0.705882  ...     SVM         0  0.516484
    26    CHEMBL2114927  fold0   0.213333  ...     SVM         0  0.455645
    27     CHEMBL763093  fold0   0.791045  ...      GC         0  0.756579
    28    CHEMBL1614525  fold0   0.482759  ...     SVM         0  0.613068
    29    CHEMBL2114803  fold0   0.739130  ...     SVM         0  0.845486
    ...             ...    ...        ...  ...     ...       ...       ...
    3900  CHEMBL1614249  fold2   0.025710  ...     SVM         0  0.626016
    3901  CHEMBL1963953  fold2   0.047646  ...     FNN         0  0.854635
    3902  CHEMBL1963854  fold2   0.045197  ...     FNN         0  0.872695
    3903  CHEMBL1963985  fold2   0.045219  ...     FNN         0  0.862658
    3904  CHEMBL1964030  fold2   0.069141  ...     FNN         0  0.818916
    3905  CHEMBL1963960  fold2   0.036632  ...     FNN         0  0.864179
    3906  CHEMBL1963929  fold2   0.047804  ...     FNN         0  0.849110
    3907  CHEMBL1964006  fold2   0.049430  ...     FNN         0  0.863670
    3908  CHEMBL1963868  fold2   0.048450  ...     FNN         0  0.840745
    3909  CHEMBL1963922  fold2   0.046376  ...     FNN         0  0.860327
    3910  CHEMBL1964077  fold2   0.046030  ...     FNN         0  0.851719
    3911  CHEMBL1964048  fold2   0.047017  ...     FNN         0  0.868728
    3912  CHEMBL1964012  fold2   0.048764  ...     FNN         0  0.861181
    3913  CHEMBL3215106  fold2   0.006904  ...     FNN         0  0.587147
    3914  CHEMBL1614087  fold2   0.719518  ...     SVM         0  0.727660
    3915  CHEMBL2114810  fold2   0.007347  ...     FNN         0  0.807068
    3916  CHEMBL1963874  fold2   0.058579  ...     FNN         0  0.852885
    3917  CHEMBL1964072  fold2   0.055914  ...     FNN         0  0.863747
    3918  CHEMBL1963981  fold2   0.044535  ...     FNN         0  0.877913
    3919  CHEMBL2354254  fold2   0.980901  ...     FNN         0  0.775291
    3920  CHEMBL1614530  fold2   0.380730  ...     SVM         